## Usando Keras para Construir e Treinar Redes Neurais

Neste exercício usaremos uma rede neural para prever diabetes usando o Dataset de Diabetes Pima. Começaremos treinando uma Random Forest para obter uma linha de base de desempenho. Em seguida, usaremos o pacote Keras para construir e treinar rapidamente uma rede neural e comparar o desempenho. Veremos como diferentes estruturas de rede afetam o desempenho, o tempo de treinamento e o nível de overfitting (ou underfitting).

## Dataset de Diabetes Pima da UCI

* Repositório ML da UCI (http://archive.ics.uci.edu/ml/datasets/Pima+Indians+Diabetes)


### Atributos: (todos com valores numéricos)
   1. Número de vezes grávida
   2. Concentração de glicose plasmática 2 horas após teste oral de tolerância à glicose
   3. Pressão arterial diastólica (mm Hg)
   4. Espessura da dobra cutânea do tríceps (mm)
   5. Insulina sérica de 2 horas (mu U/ml)
   6. Índice de massa corporal (peso em kg/(altura em m)^2)
   7. Função de pedigree da diabetes
   8. Idade (anos)
   9. Variável de classe (0 ou 1)

O Dataset de Diabetes Pima da UCI que possui 8 preditores numéricos e um resultado binário.

In [ ]:
#Preliminares

from __future__ import absolute_import, division, print_function  # Python 2/3 compatibility

import warnings
warnings.filterwarnings("ignore")


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_auc_score, roc_curve, accuracy_score
from sklearn.ensemble import RandomForestClassifier

import seaborn as sns

%matplotlib inline

In [ ]:
## Importar objetos do Keras para Deep Learning

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, SGD, RMSprop

In [ ]:
## Carregar o conjunto de dados (Acesso à Internet necessário)

url = "https://raw.githubusercontent.com/npradaschnor/Pima-Indians-Diabetes-Dataset/refs/heads/master/diabetes.csv"
names = ["times_pregnant", "glucose_tolerance_test", "blood_pressure", "skin_thickness", "insulin", 
         "bmi", "pedigree_function", "age", "has_diabetes"]
diabetes_df = pd.read_csv(url)
diabetes_df.columns = names

In [ ]:
# Dar uma olhada nos dados -- se houver muitos "NaN" você pode ter problemas de conectividade com a internet
print(diabetes_df.shape)
diabetes_df.sample(5)

In [ ]:
X = diabetes_df.iloc[:, :-1].values
y = diabetes_df["has_diabetes"].values

In [ ]:
# Dividir os dados em Treinamento e Teste (75%, 25%)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=11111)

In [ ]:
np.mean(y), np.mean(1-y)

Acima, vemos que cerca de 35% dos pacientes neste dataset têm diabetes, enquanto 65% não têm. Isso significa que podemos obter uma precisão de 65% sem nenhum modelo - apenas declarando que ninguém tem diabetes. Calcularemos a pontuação ROC-AUC para avaliar o desempenho do nosso modelo, e também observaremos a precisão para ver se melhoramos a precisão de 65%.
## Exercício: Obtenha um desempenho de linha de base usando Random Forest
Para começar e obter uma linha de base para o desempenho do classificador:
1. Treine um modelo Random Forest com 200 árvores nos dados de treinamento.
2. Calcule a precisão e o roc_auc_score das predições.

In [ ]:
## Treinar o Modelo RF
rf_model = RandomForestClassifier(n_estimators=200)
rf_model.fit(X_train, y_train)

In [ ]:
# Fazer previsões no conjunto de teste - tanto previsões "definitivas" quanto os scores (porcentagem de árvores votando sim)
y_pred_class_rf = rf_model.predict(X_test)
y_pred_prob_rf = rf_model.predict_proba(X_test)


print('accuracy is {:.3f}'.format(accuracy_score(y_test,y_pred_class_rf)))
print('roc-auc is {:.3f}'.format(roc_auc_score(y_test,y_pred_prob_rf[:,1])))

In [ ]:
def plot_roc(y_test, y_pred, model_name):
    fpr, tpr, thr = roc_curve(y_test, y_pred)
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.plot(fpr, tpr, 'k-')
    ax.plot([0, 1], [0, 1], 'k--', linewidth=.5)  # roc curve for random model
    ax.grid(True)
    ax.set(title='ROC Curve for {} on PIMA diabetes problem'.format(model_name),
           xlim=[-0.01, 1.01], ylim=[-0.01, 1.01])


plot_roc(y_test, y_pred_prob_rf[:, 1], 'RF')

## Construir uma Rede Neural de Camada Oculta Única

Usaremos o modelo Sequential para construir rapidamente uma rede neural. Nossa primeira rede será uma rede de camada única. Temos 8 variáveis, então definimos o formato de entrada como 8. Vamos começar tendo uma única camada oculta com 12 nós.

In [ ]:
## Primeiro vamos normalizar os dados
## Isso auxilia o treinamento de redes neurais fornecendo estabilidade numérica
## Random Forest não precisa disso pois encontra apenas uma divisão, ao contrário de realizar multiplicações de matrizes


normalizer = StandardScaler()
X_train_norm = normalizer.fit_transform(X_train)
X_test_norm = normalizer.transform(X_test)

In [ ]:
# Definir o Modelo 
# Tamanho de entrada é 8-dimensional
# 1 camada oculta, 12 nós ocultos, ativação sigmoid
# Camada final tem apenas um nó com ativação sigmoid (padrão para classificação binária)

model_1 = Sequential([
    Dense(12, input_shape=(8,), activation="relu"),
    Dense(1, activation="sigmoid")
])

In [ ]:
# Esta é uma ferramenta útil para visualizar o modelo que você criou e contar os parâmetros

model_1.summary()

### Pergunta de compreensão:
Por que temos 121 parâmetros? Isso faz sentido?


Vamos ajustar nosso modelo por 200 épocas.

In [ ]:
# Ajustar(Treinar) o Modelo

# Compilar o modelo com Otimizador, Função de Perda e Métricas
# Roc-Auc ainda não está disponível no Keras como uma métrica pronta para uso, então vamos pular isso aqui.

model_1.compile(SGD(learning_rate = .003), "binary_crossentropy", metrics=["accuracy"])
run_hist_1 = model_1.fit(X_train_norm, y_train, validation_data=(X_test_norm, y_test), epochs=200)
# a função fit retorna o histórico de execução. 
# É muito conveniente, pois contém informações sobre o ajuste do modelo, iterações etc.

In [ ]:
## Como fizemos para o Random Forest, geramos dois tipos de previsões
# Uma é uma decisão definitiva, a outra é um score probabilístico.

y_pred_prob_nn_1 = model_1.predict(X_test_norm)
y_pred_class_nn_1 = (y_pred_prob_nn_1 > 0.5).astype("int32")

In [ ]:
# Vamos verificar as saídas para ter uma noção de como as APIs do keras funcionam.
y_pred_class_nn_1[:10]

In [ ]:
y_pred_prob_nn_1[:10]

In [ ]:
# Imprimir performance do modelo e plotar a curva roc
print('accuracy is {:.3f}'.format(accuracy_score(y_test,y_pred_class_nn_1)))
print('roc-auc is {:.3f}'.format(roc_auc_score(y_test,y_pred_prob_nn_1)))

plot_roc(y_test, y_pred_prob_nn_1, 'NN')

Pode haver alguma variação nos números exatos devido à aleatoriedade, mas você deve obter resultados na mesma faixa da Random Forest - entre 75% e 85% de precisão, entre 0,8 e 0,9 para AUC.

Vamos observar o objeto `run_hist_1` que foi criado, especificamente seu atributo `history`.

In [ ]:
run_hist_1.history.keys()

Vamos plotar a perda de treinamento e a perda de validação ao longo das diferentes épocas e ver como fica.

In [ ]:
fig, ax = plt.subplots()
ax.plot(run_hist_1.history["loss"],'r', marker='.', label="Train Loss")
ax.plot(run_hist_1.history["val_loss"],'b', marker='.', label="Validation Loss")
ax.legend()

Parece que as perdas ainda estão diminuindo tanto no conjunto de treinamento quanto no conjunto de validação. Isso sugere que o modelo pode se beneficiar de mais treinamento. Vamos treinar o modelo um pouco mais e ver o que acontece. Observe que ele continuará de onde parou. Treine por mais 1000 épocas.

In [ ]:
## Note que quando chamamos "fit" novamente, ele continua de onde parou
run_hist_1b = model_1.fit(X_train_norm, y_train, validation_data=(X_test_norm, y_test), epochs=1000)

In [ ]:
n = len(run_hist_1.history["loss"])
m = len(run_hist_1b.history['loss'])
fig, ax = plt.subplots(figsize=(16, 8))

ax.plot(range(n), run_hist_1.history["loss"],'r', marker='.', label="Train Loss - Run 1")
ax.plot(range(n, n+m), run_hist_1b.history["loss"], 'hotpink', marker='.', label="Train Loss - Run 2")

ax.plot(range(n), run_hist_1.history["val_loss"],'b', marker='.', label="Validation Loss - Run 1")
ax.plot(range(n, n+m), run_hist_1b.history["val_loss"], 'LightSkyBlue', marker='.',  label="Validation Loss - Run 2")

ax.legend()

Observe que este gráfico começa onde o outro parou. Embora a perda de treinamento ainda esteja diminuindo, parece que a perda de validação se estabilizou (ou até piorou!). Isso sugere que nossa rede não se beneficiará de mais treinamento. Qual é o número apropriado de épocas?

## Exercício
Agora é sua vez. Faça o seguinte nas células abaixo:
- Construa um modelo com duas camadas ocultas, cada uma com 6 nós
- Use a função de ativação "relu" para as camadas ocultas, e "sigmoid" para a camada final
- Use uma taxa de aprendizado de 0,003 e treine por 1500 épocas
- Faça um gráfico da trajetória das funções de perda, precisão tanto no conjunto de treino quanto no de teste
- Plote a curva ROC para as predições

Experimente com diferentes taxas de aprendizado, números de épocas e estruturas de rede

In [ ]:
# Digite seu código aqui com as camadas 1,2 tendo ativação relu e camada 3 com ativação sigmoid

In [ ]:
# Digite seu código aqui para plotar a perda, acurácia e curva ROC